In [4]:
from dataset.data import train_data
from dataset.preprocess_v2 import X_sec_back_t, y_sec_back_t
from dataset.simulation_v2 import MLPModel
import torch
import torch.nn.functional as F
import pandas as pd


In [ ]:
sec_back_t_model = MLPModel.load_from_checkpoint('lightning_logs/split-back/checkpoints/epoch=3060-step=339771.ckpt')

sec_back_t_model.eval()
with torch.no_grad():    
    sec_back_t_pred = sec_back_t_model(X_sec_back_t.to(sec_back_t_model.device)).cpu()

print(F.mse_loss(sec_back_t_pred, y_sec_back_t) / len(y_sec_back_t))

sec_back_t_pred = sec_back_t_pred.numpy()

pd.DataFrame({
    'sec_back_t': y_sec_back_t.flatten(),
    'sec_back_t_pred': sec_back_t_pred.flatten()
}, index=train_data.index).plot(
    figsize=(15, 4),
    xlim=('2022-1-1', '2022-1-2'),
    ylim=(20, 40),
    grid=True,
)
